In [1]:
# efsyn


from bs4 import BeautifulSoup
import requests
import time

# Get base URL and tag name from user input
url = input('give the URL please')
url = url + '?_wrapper_format=html&page=0'  # Append necessary parameters for pagination
tname = input('print the tag name')

# Construct output filenames
a = 'Efsyn'
b = '_Wdate'
csv = '.csv'
simple = a + tname + csv
wd = a + tname + b + csv

# Set the threshold date; articles older than this will be skipped
searching_until = 20241231

# Function to update the page number in the URL for pagination
def new_url(local_url):
    global url
    x = local_url.find('=')  # Finds the first "="
    x = x + 1
    u = local_url.find("=", x)  # Finds the second "="
    curr_page_nr = local_url[u+1:]  # Extract current page number
    next_page_nr = int(curr_page_nr) + 1  # Increment it
    next_page_nr = str(next_page_nr)
    url = local_url.replace(curr_page_nr , next_page_nr)  # Replace old page number with new

# Get the initial page content and collect initial article URLs
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
main_content = soup.find('section' , class_ = "split-content__main")
suggested_articles = main_content.findAll('article')

# List to collect article URLs
article_url_list = []
for article in suggested_articles:
    a = article.find('a')
    article_url = a.get('href')
    article_url_list.append(article_url)

# Lists to store article metadata
date_list = []
url_list = []
tag_list = []

# Function to extract tags from an article
def tag_function(inside):
    global tag_list
    inside_tag_list = []
    page = requests.get(inside)
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        tag_section = soup.find('div' , class_ = 'article__tags')  # Find the tags section

        # Loop through each list item to get tags
        for i in tag_section("li"):
            b = i.find('a')
            tag = b.get_text()
            inside_tag_list.append(tag)

        tag_list.append(inside_tag_list)
    except:
        print('Please Check the URL. I found a broken article url: ' , inside)

# This will be filled again during iteration
article_url_list = []

# Function to extract article URLs and their dates
def url_function(url):
    global tag_list
    global searching_until
    global date_list
    global article_url_list
    global nr_of_art

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    main_content = soup.find('section' , class_ = "split-content__main")
    suggested_articles = main_content.findAll('article')

    https = "https://www.efsyn.gr"

    for article in suggested_articles:
        b = article.find('time')
        article_time = b.get('datetime')
        article_time = article_time[:10].replace('-' , '')  # Convert to yyyymmdd
        article_time = int(article_time)

        if article_time < searching_until:
            print('article time<searching until')
            flag = False  # Stop the main loop
        else:
            nr_of_art += 1
            date_list.append(article_time)
            a = article.find('a')
            article_url = a.get('href')

            if article_url.find(https) < 0:
                article_url = https + article_url  # Complete relative URL

            article_url_list.append(article_url)

# Begin main scraping loop
n = 0
flag = True
nr_of_pages = 0

while flag:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    main_content = soup.find('section' , class_ = "split-content__main")
    suggested_articles = main_content.findAll('article')

    nr_of_art = 0
    url_function(url)

    if nr_of_art != 16:  # Assumes 16 articles per page; stop if fewer found
        flag = False
    else:
        nr_of_pages += 1
        new_url(url)  # Move to next page

print('the number of the urls is ' , len(url_list))
print('the number of the pages opened is ' , nr_of_pages)

# Loop through each article URL and extract tags
for i in article_url_list:
    time.sleep(0.5)  # Pause to be polite to the server
    print('Im inside the article ' , i)
    try:
        tag_function(i)
    except:
        print('error ocured. Connection propably')

print('termatise')  # Means "terminated"

# Print various summaries
print('the date list is ' , date_list)
print('the length of the date list is ' , len(date_list))
print('the length of the url list is ' , len(article_url_list))
print('the length of the tag list is ' , len(tag_list))
print('the date of the last article is ' , date_list[-1])

import pandas as pd

# Flatten the nested tag list into one list of all tags
new_tag_list = []
for i in range(len(tag_list)):
    for j in range(len(tag_list[i])):
       new_tag_list.append(tag_list[i][j])

# Count occurrences of each tag
res = {}
for i in new_tag_list:
    res[i] = new_tag_list.count(i)

print(res)

# Convert dictionary to list of [tag, count] pairs
resultList = new_list = list(map(list, res.items()))
print(resultList)

# Save tag frequency to CSV
df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)

# Create a human-readable date format list (dd.mm.yyyy)
ddate_list = date_list
ttag_list = tag_list
ndlist = []

for i in ddate_list:
    i = str(i)
    y = i[:4]
    m = i[4:6]
    d = i[6:]
    ddate = d + '.' + m + '.' + y
    ndlist.append(ddate)

print(ddate_list)
print(ndlist)

# Create a list of [tag, date] pairs
n = 0
date_con_list = []
for i in ttag_list:
    n += 1
    for tag in i:
        inside = []
        inside.append(tag)
        inside.append(ddate_list[n - 1])
        date_con_list.append(inside)

print(date_con_list)

# Save tag-date associations to CSV
df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)

# Final summaries
print(len(tag_list))
print(len(date_list))
print(tag_list)
print('the last date collected is ' , date_list[-1])
print('the nr of the tags mentioned is ' , len(resultList))


article time<searching until
article time<searching until
article time<searching until
article time<searching until
article time<searching until
article time<searching until
article time<searching until
article time<searching until
article time<searching until
article time<searching until
article time<searching until
article time<searching until
article time<searching until
article time<searching until
the number of the urls is  0
the number of the pages opened is  4
Im inside the article  https://www.efsyn.gr/politiki/475456_opinion-poll-poy-simeionei-protia-i-pleysi-eleytherias
Im inside the article  https://www.efsyn.gr/politiki/474916_dimoskopisi-oi-polites-krinoyn-arnitika-tin-kybernisi-kai-ergo-tis
Im inside the article  https://www.efsyn.gr/politiki/paraskinia/474522_kybernitikoi-lagoi
Im inside the article  https://www.efsyn.gr/politiki/474343_dimoskopisi-posoi-tha-psifizan-ena-komma-toy-alexi-tsipra
Im inside the article  https://www.efsyn.gr/politiki/paraskinia/474209_hastoyk